<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/Keras%E7%94%9F%E6%80%81%E7%B3%BB%E7%BB%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras调优器
### KerasTuner是一个易于使用、可拓展的超参数优化框架，解决了超参数搜索的痛点。

## Keras自然语言处理
### KerasNLP是一个自然语言处理库，可为用户的整个开发周期提供支持。

## KerasCV
### KerasCV是一个模块化构建块(层、指标、损失、数据增强)的存储库，应用计算机视觉工程师可以利用它为常见用例快速组装生产级、最先进的训练和推理管道，例如图像分类、目标检测、图像分割、图像数据增强等。

## AutoKeras
### AutoKeras是一个基于Keras的AutoML系统。目标是让每个人都能使用机器学习。

In [ ]:
# AutoKeras
# import autokeras as ak

# clf = ak.ImageClassifier()
# clf.fit(x_train, y_train)
# results = clf.predict(x_test)

## 模型优化工具包
### 其是一组实用程序，可通过执行训练后权重量化和剪枝感知训练，使推理模型更快、内存效率更高、能效更高。它对Keras模型具有本机支持，并且其修建API直接构建在Keras API之上

In [ ]:
# import tensorflow_model_optimization as tfmot

# Define a Keras model.
# model = tf.keras.Sequential([...])

# Define a training-time pruning schedule
# pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
#     initial_sparsity=0.0, final_sparsity=0.5,
#     begin_step=2000, end_step=4000)

# Convert your Keras model to a pruning-optimized model
# model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(
#     model, pruning_schedule=pruning_schedule)

# Fit the optimized model
# model_for_pruning.fit(...)

# 一般的问题

## 如何在多个GPU上(在一台机器上)训练Keras模型
### 在多个GPU上运行单个模型有两种方法：数据并行和设备并行。在大多数情况下，您最需要的是数据并行性
### 1）数据并行性
数据并行性在于在每个设备上复制一次目标模型，并使用每个副本来处理输入数据的不同部分。

In [ ]:
# a) 实力化"分发策略"对象，例如MirroredStrategy（在每个可用设备上复制模型并保持每个模型的状态同步
# strategy = tf.distribute.MirroredStrategy()
# b） 创建模型并在策略范围内编译它
# with strategy.scope():
#   # This could be any kind of model -- Functional, subclass...
#   model = tf.keras.Sequential([
#       tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
#       tf.keras.layers.GlobalMaxPooling2D(),
#       tf.keras.layers.Dense(10)])
#   model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#          optimizer=tf.keras.optimizers.Adam(),
#          metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# model.fit(train_dataset, epochs=12, callbacks=callbacks)

### 2) 模型并行性
模型并行性在于在不同设备上运行同一模型的不同部分。它最适合具有并行架构的模型，例如具有两个分支的模型

In [ ]:
# Model where a shared LSTM is used to encode two different sequences in parallel
# input_a = keras.Input(shape=(140, 256))
# input_b = keras.Input(shape=(140, 256))

# shared_lstm = keras.layers.LSTM(64)

# Process the first sequence on one GPU
# with tf.device_scope('/gpu:0'):
#   encode_a = shared_lstm(input_a)

# Process the next sequence on another GPU
# with tf.device_scope('/gpu:1'):
#   encode_b = shared_lstm(input_b)

# Concatenate results on CPU
# with tf.device_scope('cpu:0'):
#   merged_vector = keras.layers.concatenate(
#       [encoded_a, encoded_b], axis=-1)

## 保存模型
### 不建议使用pickle或cPickle保存Keras模型
1，全模型保存（配置+权重）
保存：model.save(your_file_path.keras)
加载：model = keras.models.load_model(your_file_path)

In [1]:
# 自定义train_step
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [5]:
class MyCustomModel(keras.Model):

  def train_step(self, data):

    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)
      loss = self.compiled_loss(y, y_pred,
                      regularization_losses=self.losses)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    self.compiled_metrics.update_state(y, y_pred)

    return {m.name: m.result() for m in self.metrics}

In [6]:
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = MyCustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [7]:
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 1s 2ms/step - loss: 0.3057 - accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.2638 - accuracy: 0.0000e+00
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.2536 - accuracy: 0.0000e+00
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2432 - accuracy: 0.0000e+00
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2329 - accuracy: 0.0000e+00
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2221 - accuracy: 0.0000e+00
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2124 - accuracy: 0.0000e+00
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.2030 - accuracy: 0.0000e+00
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1944 - accuracy: 0.0000e+00
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1861 - 

In [12]:
# 还可以轻松添加对样本加权的支持
class MyCustomModel(keras.Model):

  def train_step(self, data):

    if len(data) == 3:
      x, y, sample_weight = data
    else:
      x, y = data

    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)

      loss = self.compiled_loss(y, y_pred,
                      sample_weight=sample_weight,
                      regularization_losses=self.losses)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    self.compiled_metrics.update_state(
        y, y_pred, sample_weight=sample_weight)
    return {m.name: m.result() for m in self.metrics}

In [13]:
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = MyCustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2295 - accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1521 - accuracy: 0.0000e+00
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1444 - accuracy: 0.0000e+00
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1404 - accuracy: 0.0000e+00
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1354 - accuracy: 0.0000e+00
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1315 - accuracy: 0.0000e+00
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1263 - accuracy: 0.0000e+00
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1214 - accuracy: 0.0000e+00
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1168 - accuracy: 0.0000e+00
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1123 - 

In [14]:
# 还可以通过覆盖来自定义评估test_step
class MyCustomModel(keras.Model):
  def test_step(self, data):
    x, y = data
    y_pred = self(x, training=False)
    self.compiled_loss(
        y, y_pred, regularization_losses=self.lossess)
    self.compiled_metrics.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.kl_divergence

for inputs, target in dataset:
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss_value = loss_fn(targets, predictions)
  gradients = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))

NameError: ignored

## 如何获得中间层的输出(特征提取)

In [16]:
# 在函数式API中，如果某个层仅被调用一次，可以通过检索输出layer.output和其输入layer.input

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = Sequential([
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10),
])
extractor = keras.Model(inputs=model.inputs,
             outputs=[layer.output for layer in model.layers])
features = extractor(data)

In [17]:
# 使用有状态的RNN

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

x = np.random.random((32, 21, 16))
model = keras.Sequential()
model.add(layers.LSTM(32, input_shape=(10, 16), batch_size=32, stateful=True))
model.add(layers.Dense(16, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.train_on_batch(x[:, :10, :], np.reshape(x[:, 10, :], (32, 16)))

model.train_on_batch(x[:, 10:20, :], np.reshape(x[:, 20, :], (32, 16)))

model.reset_states()

model.layers[0].reset_states()